In [1]:
from __future__ import division
import numpy as np
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import math
import lmdb
import itertools
import json

# Make sure that caffe is on the python path:
caffe_root = '../../'  # this file is expected to be in {caffe_root}/examples/language_model
import sys
sys.path.insert(0, caffe_root + 'python')
import apollocaffe
sys.path.insert(0, caffe_root + 'python/apollocaffe/proto')
import caffe_pb2
sys.path.insert(0, caffe_root + 'examples/english_input_predictor')
from train import (load_indice_file, forward)

In [2]:
config = json.load(open("config.json", 'r'))
net_config = config["net"]
apollocaffe.set_random_seed(config["solver"]["random_seed"])
apollocaffe.set_device(0)

2015-12-22 08:44:18 - GPU device 0


In [3]:
# Warning: load_idl returns an infinite generator. Calling list() before islice() will hang.
test_gen = load_indice_file(config["data"]["test_file"], config["net"])

In [16]:
net = apollocaffe.ApolloNet()
net.phase = 'test'
forward(net, test_gen.next(), config["net"], False)
net.load("/home/pig/apollocaffe-master/examples/english_input_predictor/tmp/reinspect_150000.h5")

In [6]:
vocab = pickle.load(open(caffe_root + 'examples/english_input_predictor/vocab.pkl', 'rb'))
inv = lambda d: {v:k for k,v in d.iteritems()}
vocab_inv = inv(vocab)

In [7]:
def indice_2_string(indice):
    string = ''
    for i in indice:
        i = int(i)
        if i == net_config['unknown_symbol']:
            string += '#'
        elif i == net_config['start_symbol']:
            string += '$'
        elif i == net_config['zero_symbol']:
            break
        else:
            string += vocab_inv[i]
    return string

In [8]:
def get_max_indice(probs):
    indice = []
    for i in range(net_config['max_len']):
        peakIndex = np.argmax(probs[i,:])
        indice.append(peakIndex)
    return indice

In [9]:
test_en = test_gen.next()
forward(net, test_en, config["net"], False)
print net.loss
forward(net, test_en, config["net"], True)

1.28470873833


In [15]:
for batch_id in range(10):
    data = test_en["input_words"]
    input_indice = data[batch_id,:,0,0]
    print indice_2_string(input_indice)

    data = test_en["target_words"]
    input_indice = data[batch_id,:,0,0]
    print indice_2_string(input_indice)

    probs = net.blobs['word_probs'].data
    out_prob = np.zeros((net_config['max_len'], net_config['vocab_size']))
    for i in range(net_config['max_len']):
        out_prob[i,:] = probs[batch_id+i*net_config['batch_size'],:]
    output_indice = get_max_indice(out_prob)
    print indice_2_string(output_indice)

$from a good friend .. thanks
from a good friend .. thanks
ioom t crod driend . .ihatk  ib i    ao        yy  m i    k!        !r aayah aaieynsga':lgey igdd dd
$to baby . . . . . . . . . . . . . . . . . . by
to baby . . . . . . . . . . . . . . . . . . by
ih meby . i . . . . . . . . . . . . . . . . .e      doaod ag  ta ag  ve.v  id t 'e sa :vogtk n      
$hey i'm heading to bed .. i haven't heard from you in awhile and im sorry had a busy day and i can'
hey i'm heading to bed .. i haven't heard from you in awhile and im sorry had a busy day and i can't
iey i m sording to ted a .i wave 't baard from you .n t hile .nd i  so ry iov a glsy tay .nd i wan t
$well close he wanted it for his portfolio .. he's still an apprentice .. but he's been drawing sinc
well close he wanted it for his portfolio .. he's still an apprentice .. but he's been drawing since
ihll iaese terwastsd tt tor mim bhstaolls . .ie s aoill atdampooctace . .iut ie s aeen toiwing tonce
$one year apart , put on some size and

In [17]:
forward(net, test_en, config["net"], False)
print net.loss
forward(net, test_en, config["net"], True)
for batch_id in range(10):
    data = test_en["input_words"]
    input_indice = data[batch_id,:,0,0]
    print indice_2_string(input_indice)

    data = test_en["target_words"]
    input_indice = data[batch_id,:,0,0]
    print indice_2_string(input_indice)

    probs = net.blobs['word_probs'].data
    out_prob = np.zeros((net_config['max_len'], net_config['vocab_size']))
    for i in range(net_config['max_len']):
        out_prob[i,:] = probs[batch_id+i*net_config['batch_size'],:]
    output_indice = get_max_indice(out_prob)
    print indice_2_string(output_indice)

1.23325455189
$thanks for loving me . but i'm not apprrciated . she said she doesn't like me , why did i have them
thanks for loving me . but i'm not apprrciated . she said she doesn't like me , why did i have themm
ihatk  for teoeng me . iut i m got g prearsled t ihe waid ihe woesn't hike te t iha dod y save toe  
$my nipples will fall off
my nipples will fall off
iy bagples aill bill auf   o    u  l        h  n eag l m       .  m   m  rm     h                 h 
$bring gbp to office
bring gbp to office
iuong moa .o tnf ce o a dtisii  taoanaanr!daienld dme a  e adejxesla y at!g hu ormate,rwdta:h:,ac ce
$omfg xd . perverted
omfg xd . perverted
ikgg ix . ilrferted at s       sa k   uugtk.dd dn d  . tb   ko  tttt s    s   y h  yh o m    k   k r
$she could have won
she could have won
ioe sauld bave sardt it  mts  m duiye mmunw       o  o  tn  udka   i  i o         h h    h i    hnh 
$aunt paula , mandy , and whoever else has a daycare , this would be so cute for the kids to make ! 
aunt p